# How to start

Before starting you must:
- Ensure that `scipp` and `mantid` are on your `PYTHONPATH`.
- Generate the `config.py` file using `make_config.py`. Refer to the `README.md` or `python make_config.py --help` for information.
- Install dependencies : `conda install fabio tifffile` (used for image handling)

For `scipp` and `mantid` follow instructions at: https://scipp.readthedocs.io/en/latest/getting-started/installation.html.

Converted to use scipp and notebook from [this repistory](https://git.esss.dk/testbeamline/gp2/-/tree/stressexperiment).


In [ ]:
try:
    import scipp
except ImportError as e:
    print("scipp is not available in the PYTHONPATH")
    raise e
    
try:
    import mantid
except ImportError as e:
    print("mantid is not available in the PYTHONPATH")
    raise e
    
try:
    import scippconfig
except ImportError as e:
    print("scippconfig is not available. Make sure you have generated it with `make_config.py`.")
    raise e

In [ ]:
# Lets get everything set up
from multiprocessing.dummy import Pool as ThreadPool
import os
import sys

import scipp as sc
import numpy as np

import imaging, operations

from scipy import ndimage, signal

DATA_DIR_NAME = "data_GP2"
OUTPUT_DIR_NAME = "output"

data_dir = os.path.join(scippconfig.script_root, DATA_DIR_NAME)
output_dir = os.path.join(scippconfig.script_root, OUTPUT_DIR_NAME)
instrument_file = os.path.join(scippconfig.script_root, 'IDF', 'V20_Definition_GP2.xml')

if not os.path.exists(data_dir):
    raise FileNotFoundError("The following data directory does not exist,"
                            f" check your make_config.py:\n{data_dir}")

In [ ]:
# Customisable Options:

# Whether or not to do the plotting.
do_plots = True

# defining grouping of 2D detector pixels
grouping_number = 27
nx_target = grouping_number
ny_target = grouping_number

# Rebin regions for each of the 5 frames
# in the format of [bin-start, bin-end, bin width].
# used to crop each image, before stitching them together
frame_parameters = [(15450, 22942, 64),
                    (24800, 32052, 64),
                    (33791, 40084, 64),
                    (41763, 47457, 64),
                    (49315, 54500, 64),
                    (56500, 58360, 64)]

# Used to shift the cropped frames so that their bins overlap 
# before summing them together into a single frame
frame_shift_increments = [-6630, -2420, -2253, -2095, -1946, -1810]
frame_shift_increments = [float(i) for i in frame_shift_increments]  # Work around #1114

# Used to rebin the summed frame in order to
# cut off frames that contain no data
rebin_parameters = {"start": 8550, "stop": 26000, "width": (64 * 2.5)}

# Pulse references
pulse_number_reference = 1.0 / 770956
pulse_number_sample = 1.0 / 1280381
pulse_number_sample_elastic = 1.0 / 2416839
pulse_number_sample_plastic = 1.0 / 2614343

# units of transmission, all pixels with transmission higher masking threshold are masked
masking_threshold = 0.80

# Toggles outputting masked and sliced tiff stacks
output_tiff_stack = True

# Experiment Metadata
measurement_number = 11

In [ ]:
frame_shifts = [sum(frame_shift_increments[:i + 1]) for i in range(len(frame_shift_increments))]

# let's get the process started:
tofs_path = os.path.join(data_dir, 'metadata', 'GP2_Stress_time_values.txt')
raw_data_dir = os.path.join(data_dir, "Stress_Experiments")

ds = sc.Dataset()
# Load X values from the TOF file
ds.coords["tof"] = sc.Variable(["tof"], unit=sc.units.us, values=imaging.read_x_values(tofs_path))
ds.coords["tof"] *= 1e3

def load_and_scale(folder_name, scale_factor):
    to_load = os.path.join(raw_data_dir, folder_name)
    variable = imaging.tiffs_to_variable(to_load)
    variable *= scale_factor
    return variable

ds["reference"] = load_and_scale(folder_name="1) R825 Open Beam", scale_factor=pulse_number_reference)
ds["sample"] = load_and_scale(folder_name="2) R825", scale_factor=pulse_number_sample)
ds["sample_elastic"] = load_and_scale(folder_name="3) R825 600 Mpa", scale_factor=pulse_number_sample_elastic)
ds["sample_plastic"] = load_and_scale(folder_name="4) R825 3500 um", scale_factor=pulse_number_sample_plastic)

In [ ]:
# Adds a coordinate for the spectra and TOF
ds.coords["spectrum"] = sc.Variable(["spectrum"], values=np.arange(ds["sample"].shape[1]))
stitched = sc.Dataset(coords={"tof": sc.Variable(["tof"], unit=sc.units.us,
                              values=np.arange(start=rebin_parameters["start"], stop=rebin_parameters["stop"],
                                               step=rebin_parameters["width"], dtype=np.float64))})

In [ ]:
def stitch_data(variable_to_stitch):
    print(f"Stitching: {variable_to_stitch}")
    stitched[variable_to_stitch] = imaging.stitch(ds[variable_to_stitch],
                                                  frame_parameters=frame_parameters,
                                                  frame_shifts=frame_shifts,
                                                  rebin_parameters=rebin_parameters)

for var_name in ds.keys():
    stitch_data(var_name)

In [ ]:
tof = stitched.coords['tof']
bin_widths = tof['tof', 1:] - tof['tof', :-1]

integrated = sc.Dataset()
for dim in stitched.coords.keys():
    integrated.coords[dim] = stitched.coords[dim]

integrated = sc.sum(stitched, 'tof')
# Put TOF coords back for the export to TIFF stack later
integrated.coords["tof"] = stitched.coords["tof"]

In [ ]:
num_spectra = stitched.coords["spectrum"].shape[0]
bank_width = int(np.sqrt(num_spectra))

In [ ]:
# Pull out reference from out dataset to avoid checking for it in loops
reference = stitched["reference"]
integrated_reference = integrated["reference"]

# Causes segmentation fault, see scipp/scipp issue: #1176
# del stitched["reference"]
# del integrated["reference"]


In [ ]:
def mask_non_sample_regions(integrated_dataset, var_name):
    integrated_spectra = integrated_dataset[var_name]

    # This should use sc.greater once #1178 is completed
    spectra_masks = np.greater(integrated_spectra.values, masking_threshold)
    spectra_masks = sc.Variable(["spectrum"], values=spectra_masks)
    final_mask = operations.mask_from_adj_pixels(spectra_masks, bank_width)
    stitched.masks[var_name] = final_mask


integrated /= integrated_reference

for k in integrated.keys():
    if k == "reference":  # This can be removed once #1176 is resolved
        continue

    print(f"Masking non-sample regions in {k}")
    mask_non_sample_regions(integrated, k)

    if output_tiff_stack:
        print(f"Exporting tiff stack for {k}")
        tiff_out_dir = os.path.join(output_dir, f"tiffs_tof_sum_{k}")
        imaging.export_tiff_stack(dataset=integrated, key=k,
                                  base_name=f"{k}_norm_sum",
                                  output_dir=tiff_out_dir,
                                  x_len=324, y_len=324)
        # Print a newline to prevent the saving messages overlapping
        print()

In [ ]:
# Normalize by open beam
normalized = stitched / reference

# Replace special values nan and inf
replacement=sc.Variable(value=0.0, variance=0.0)
kwargs = {"nan" : replacement, "posinf" : replacement, "neginf" : replacement}
for k in normalized.keys():
    sc.nan_to_num(normalized[k].data, out=normalized[k].data, **kwargs)

In [ ]:
if output_tiff_stack:
    imaging.export_tiff_stack(dataset=normalized, key="sample", x_len=324, y_len=324,
                              base_name="initial_tof", output_dir=os.path.join(output_dir, "tiffs_tof_initial"))
    imaging.export_tiff_stack(dataset=normalized, key="sample_elastic", x_len=324, y_len=324,
                              base_name="elastic_tof", output_dir=os.path.join(output_dir, "tiffs_tof_elastic"))
    imaging.export_tiff_stack(dataset=normalized, key="sample_plastic", x_len=324, y_len=324,
                          base_name="elastic_tof", output_dir=os.path.join(output_dir, "tiffs_tof_plastic"))

In [ ]:
sc.compat.mantid.load_component_info(normalized, instrument_file)
wavelength = sc.neutron.convert(normalized, "tof", "wavelength")

# Position data ...etc. is dropped on conversion but is later required for plotting
sc.compat.mantid.load_component_info(wavelength, instrument_file)

In [ ]:
# Apply a median filter to the sample after converting to wavelength
apply_median_filter = True

def median_filter(dataset, variable_key):
    print(f"Running median of workspace {variable_key}")
    data = dataset[variable_key].values
    masks = dataset.masks[variable_key].values

    # Repeat does not allocate mem, and makes it easier to double check
    # masking is being handled properly using debugging tooling
    mask = np.repeat(masks[np.newaxis, :], data.shape[0], axis=0)

    masked_data = np.ma.masked_array(data, mask=mask)

    # The number
    result = ndimage.median_filter(masked_data,
                                   size=(data.shape[0], 3),  # Take all wavelength vals for 2 adj spectra
                                   mode="constant", cval=0.)  # Set any vals outside median window to 0.
    dataset[variable_key].values = result

if apply_median_filter:
    for k in wavelength.keys():
        if k == "reference":  # This can be removed once #1176 is resolved
            continue

        new_result = median_filter(dataset=wavelength, variable_key=k)

    # TODO export_to_tiff_stack for each

In [ ]:
wav_x_vals = wavelength["sample"].coords["wavelength"].values
start_wave = np.max(wav_x_vals[:, 0])
stop_wave = np.min(wav_x_vals[:, -1])
bin_width_wave = np.mean(wav_x_vals[:, 1:]-wav_x_vals[:, :-1])
wavelength_rebin_params = sc.Variable(["wavelength"], values=np.arange(start_wave, stop_wave, bin_width_wave,
                                                           dtype=np.float64))

rebinned_wavelength = sc.rebin(wavelength, "wavelength", wavelength_rebin_params)

# if True, the whole initial state is integrated, if False initial state is also grouped
integrating_of_sample_in_inital_state = True
if integrating_of_sample_in_inital_state:
    group_size = (324.0  / nx_target) * (324.0 / ny_target)
    factor = group_size
    rebinned_wavelength["sample"] *= factor

In [ ]:
def weighted_sum(variable : sc.Variable, dim_to_sum):
    variable = variable.copy()

    variances = sc.Variable(dims=variable.dims, values=variable.variances)
    d = sc.Dataset()
    d['data'] = variable / variances
    d['norm'] = sc.reciprocal(variances)

    d.masks['zero_error'] = sc.equal(variances, 0.0 * variances.unit)
    
    d = sc.sum(d, dim_to_sum)
    sum = d['data']/d['norm']
    return sum

# If we are not interested in strain regions for the unloaded sample, we can just combine all spectra to improve statistics.
# maybe another masking before summation, with tougher threshold to exclude boderline pixels.
rebinned_wavelength["sample"] = weighted_sum(rebinned_wavelength["sample"], "spectrum")

In [ ]:
rebinned_wavelength.coords["detector_mapping"] = imaging.make_detector_groups(324, 324, nx_target, ny_target)
grouped = sc.groupby(rebinned_wavelength, "detector_mapping").sum("spectrum")
grouped_sample_elastic = grouped["sample_elastic"]

In [ ]:
histogram_matrices = [rebinned_wavelength["sample"], grouped_sample_elastic]
template_histogram = histogram_matrices[1]

# Average out detector positions
positions_dataset = sc.Dataset()
positions_dataset["position"] = sc.Variable(rebinned_wavelength.coords["position"])
positions_dataset.coords["detector_mapping"] = rebinned_wavelength.coords["detector_mapping"]
grouped_positions = sc.groupby(positions_dataset, "detector_mapping").mean("spectrum")

plots = []
for spec_num in template_histogram.coords["detector_mapping"].values:
    pos = grouped_positions["position"]["detector_mapping", int(spec_num)]
    # Append spectrum index and position
    plots.append((spec_num,pos))


# We cant access position using (dim, I), since the vector 3d has not dim currently
Y_key = 1
X_key = 0

# We must use round to prevent floating points errors during sorting like -0.05 > 0.05
plots.sort(key=lambda t: (round(-t[1].value[Y_key], 10), round(t[1].value[X_key], 10)))

In [ ]:
# Bragg edge position roughly, in Angstroem
FCC_a = 3.5 # Aangstroem, taken from COD entry 9008469
BCC_a = 2.865 # Aangstroem, average value from COD
indices_FCC = [(1,1,1),(2,0,0),(2,2,0),(3,1,1)]#,(2,2,2)]#,(4,0,0)]
indices_BCC = [(1,1,0),(2,0,0),(2,1,1),(2,2,0)]#,(3,1,0),(2,2,2)]

def create_Braggedge_list(lattice_constant, Miller_indices):
    '''
    :param Miller_indices: like [(1,1,0),(2,0,0),...]
    :type Miller_indices: list of tuples
    '''
    from numpy import sqrt
    return [(d ,2.*lattice_constant/sqrt(d[0]**2+d[1]**2+d[2]**2)) for d in Miller_indices]


Bragg_edges_FCC = create_Braggedge_list(FCC_a, indices_FCC)

In [ ]:
def convert_var_to_ws(variable, dim):
    x_vals = np.array(variable.coords["wavelength"].values)

    ws = sc.compat.mantid.to_workspace_2d(x=x_vals,
                                          y=np.transpose(variable.values),
                                          e=np.transpose(variable.variances),
                                          coord_dim=dim,
                                          instrument_file=instrument_file)
    return ws


def Bragg_edge_position(xpos_Bragg_edge,x_min_sides,x_max_sides,x_size,y_size,plots):
    '''Takes limits around a roughly known Bragg edge and fits the Bragg edge to obtain
    its exact position. Needs the size (x_size, y_size) and the workspaces containing histogram arrays/matrices.
    x_min_side and x_max_side are arbitrary values and are calculated:
    bragg_edge_pos * x_min_side or bragg_edge_pos * x_max_side.
    '''

    fit_list = []
    spectrum_list_fitted_sample = []
    spectrum_list_fitted_sample_elastic = []

    if not (len(xpos_Bragg_edge) == len(x_min_sides) or len(xpos_Bragg_edge) == len(x_max_sides)):
        print("xpos_Bragg_edge has not the same length as either x_min_sides or x_max_sides. Check your input.")
        return
    
    elastic_ws = convert_var_to_ws(grouped_sample_elastic, "wavelength")
    sample_ws = convert_var_to_ws(rebinned_wavelength["sample"], "wavelength")

    # loop for each Bragg edge in the list, find tof positions and save spectrums in lists
    for c, values in enumerate(xpos_Bragg_edge):
        fit_list.append([])
        spectrum_list_fitted_sample.append([])
        spectrum_list_fitted_sample_elastic.append([])
        bragg_edge = '({}{}{})'.format(values[0][0], values[0][1], values[0][2])
        xpos_guess = values[1]
        x_min_fit = xpos_guess - xpos_guess * abs(x_min_sides[c])
        x_max_fit = xpos_guess + xpos_guess * abs(x_max_sides[c])

        print("Now fitting Bragg edge {} at {:.3f} A (between {:.3f} A and {:.3f} A)".format(bragg_edge, xpos_guess, x_min_fit, x_max_fit))


        # if the full inital sample was taken and no grouping was done
        if "spectrum" not in rebinned_wavelength["sample"].coords:
            print('Sample workspace contains one histogram.')
            print(f'Elastic Workspace contains {wavelength["sample_elastic"].coords["spectrum"].shape[0]} histograms.')
            #Fitting the masked sample


            sample_fit = sc.compat.mantid.fit(sample_ws,
                                          function=f'name=LinearBackground,A0={230},A1={-4};name=UserFunction,Formula=h*erf(a*(x-x0)),h={16},a={-11},x0={xpos_guess}',
                                          workspace_index=0, start_x=x_min_fit, end_x=x_max_fit)

            params = dict(zip(sample_fit["parameters"].value["Name"].values, sample_fit["parameters"].value["Value"].values))

            # take the fitting parameters and pass them on to the forthcoming fits
            d_sample = params["f1.x0"] / 2.0 # See fit table definition for extract x0
            fit_values = (params["f0.A0"], params["f0.A1"], params["f1.h"], params["f1.a"], params["f1.x0"])


        plots_counter = 0 # sets the workspace index to zero then increases one by one
        for row in range(y_size):
            for col in range(x_size):
                ws_index = plots[plots_counter][0]

                # if the inital sample was grouped
                if "spectrum" in rebinned_wavelength["sample"].coords:

                    sample_fit = sc.compat.mantid.fit(sample_ws,
                                                  function=f"name=LinearBackground,A0={230},A1={-4};name=UserFunction,Formula=h*erf(a*(x-x0)),h={16},a={-11},x0={xpos_guess}",
                                                  workspace_index=ws_index, start_x=x_min_fit, end_x=x_max_fit)
                    
                    # take the fitting parameters and pass them on to the forthcoming fits
                    params = dict(zip(sample_fit["parameters"].value["Name"].values, sample_fit["parameters"].value["Value"].values))

                    # take the fitting parameters and pass them on to the forthcoming fits
                    d_sample = params["f1.x0"] # See fit table definition for extract x0
                    fit_values = (params["f0.A0"], params["f0.A1"], params["f1.h"], params["f1.a"], params["f1.x0"])

                # Fit Bragg edge using values from fit of unstrained sample
                elastic_function = f"name=LinearBackground,A0={fit_values[0]},A1={fit_values[1]};name=UserFunction,Formula=h*erf(a*(x-x0)),h={fit_values[2]},a={fit_values[3]},x0={fit_values[4]}"
                fit_elastic = sc.compat.mantid.fit(elastic_ws,
                                                          function=elastic_function,
                                                          workspace_index=ws_index, start_x=x_min_fit, end_x=x_max_fit)
                elastic_params = dict(zip(fit_elastic["parameters"].value["Name"].values, fit_elastic["parameters"].value["Value"].values))
                d_sample_elastic = elastic_params["f1.x0"] / 2.0 # See fit table definition for extract x0
                lattice_strain = d_sample_elastic - d_sample

                # define successful fitting
                success = (sample_fit.attrs["status"].value == "success") and (fit_elastic.attrs["status"].value == "success")

                # fitted values STORED in list
                fit_list[c].append((ws_index, (row, col), bragg_edge, d_sample, d_sample_elastic, lattice_strain, success))

                # workspace created from sample fit, workspace index 1 for fitted spectrum (index 0 for data, index 2 for difference curve)
                fitted_sample = sample_fit["workspace"].data.values["empty", 1]
                
                if sample_fit.attrs["status"].value == "success":
                    spectrum_list_fitted_sample[c].append((fitted_sample.coords["wavelength"].values, fitted_sample.values))
                else:
                    spectrum_list_fitted_sample[c].append((fitted_sample.coords["wavelength"].values,
                                                           np.zeros_like(fitted_sample.values)))
                    
                fitted_elastic = fit_elastic["workspace"].data.values["empty", 1]

                # workspace created from fit of sample under elastic deformation
                if fit_elastic.attrs["status"].value == "success":
                    spectrum_list_fitted_sample_elastic[c].append((fitted_elastic.coords["wavelength"].values, fitted_elastic.values))
                else:
                    spectrum_list_fitted_sample_elastic[c].append((fitted_elastic.coords["wavelength"].values, 
                                                                                               np.zeros_like(fitted_elastic.values)))
                plots_counter+=1

    return fit_list, spectrum_list_fitted_sample, spectrum_list_fitted_sample_elastic

fit_list, spectrum_list_fitted_sample, spectrum_list_fitted_sample_elastic = \
    Bragg_edge_position(xpos_Bragg_edge=Bragg_edges_FCC,
                                      x_min_sides=[0.05, 0.1, 0.1, 0.05],#[0.1, 0.1, 0.1, 0.05]
                                      x_max_sides=[0.1, 0.05, 0.1, 0.1],#[0.1, 0.1, 0.1, 0.1]
                                      x_size=nx_target,
                                      y_size=ny_target, 
                                      plots=plots)

In [ ]:
def write_ws_to_ascii(input_list, output_filename_stem, output_directory):
    '''
    Takes histogram from mantid workspace and writes an ASCII file.
    New filename will be the filename of the input workspace and
    a given extension appended.

    :param input_ws: input workspace from mantid
    :param output_directory: Where to write the new file
    :param output_extension: which extension should the ASCII file have

    :raises: Error if file exists
    '''

    output_filenames = []
    for c, li in enumerate(input_list):
        bragg_edge = '({}{}{})'.format(li[0][2][1], li[0][2][2],li[0][2][3])
        output_filename = "_{}_{:.3f}A_{}.txt".format(output_filename_stem, li[0][3], bragg_edge)
        if not isinstance(li, list):
            print("Input is not a list.")
            return
        whole_path = os.path.join(output_directory,output_filename)
        if os.path.isfile(whole_path):
            sys.stderr.write("ERROR: file {:s} already exists!\n".format(output_filename))
            return
        with open(whole_path, 'w') as output_file:
            for line in li:
                for element in line:
                    if not element == line[-1]:
                        output_file.write("{}\t".format(element))
                    else:
                        output_file.write("{}\n".format(element))
        print("File {} was created.".format(output_filename))
        output_filenames.append(output_filename)
    return output_filenames

tbin_width = 64*2.5
output_filename_table_stem = '{:03d}_table_strain_analysis_{}xy_{}usbin_{}thresh_initintegr{}'.format(
                        measurement_number,grouping_number,tbin_width,masking_threshold, str(integrating_of_sample_in_inital_state))



write_ws_to_ascii(fit_list, output_filename_table_stem, output_dir)

In [ ]:
import matplotlib.pyplot as plt

def tileplot_colorcode(fit_list, nx_target, ny_target, outlier_threshold, output_filename_stem='tileplot_color'):
    # for each part of the fit list containing the fit values of each Bragg edge, create a color plot.
    output_filename_list = []
    for fit_be_list in fit_list:
        bragg_edge = '({}{}{})'.format(fit_be_list[0][2][1],fit_be_list[0][2][2],fit_be_list[0][2][3])  # the calculated Bragg edge TOF position
        d_spacing = fit_be_list[0][3] # the calculated Bragg edge TOF position
        output_filename = '{}_{:.3f}A_{}'.format(output_filename_stem, d_spacing, bragg_edge)
        # Make a 2D image of the strain values
        print(('Plotting color-coded tile plot of Bragg edge {}.'.format(bragg_edge)))
        fig, ax = plt.subplots()
        strains = np.zeros([ny_target, nx_target])
        plots_counter = 0
        for row in range(ny_target):
            for col in range(nx_target):
                if (fit_be_list[plots_counter][-1] and outlier_threshold[0] < fit_be_list[plots_counter][-2] < outlier_threshold[1]):
                    strains[row, col] = fit_be_list[plots_counter][-2]
                plots_counter += 1
        import matplotlib.colors as colors
        im = ax.imshow(strains, origin="upper", norm=colors.SymLogNorm(linthresh=1.0e-3), cmap="RdBu")
        cb = plt.colorbar(im)
        cb.set_label("${}$ Lattice strain $\\varepsilon$".format(bragg_edge))
        print(f'Saving {output_filename}.pdf and .png.')
        fig.savefig("{}.pdf".format(output_filename), bbox_inches="tight")
        fig.savefig("{}.png".format(output_filename), bbox_inches="tight")
        output_filename_list.append(output_filename)
    return output_filename_list



# i.e. -1e-2, 1e-2 means only -1% to 1% values of lattice strain are shown
outlier_threshold_color_plot = (-5e-2, 5e-2)

output_filename_tileplot_color ='{:03d}_tileplot_color_{}xy_{}usbin_{}thresh_{:.3f}-{:.3f}plotthresh_initintegr{}'.format(
                        measurement_number,grouping_number,tbin_width,masking_threshold,
                        outlier_threshold_color_plot[0], outlier_threshold_color_plot[1],
                        str(integrating_of_sample_in_inital_state))

fignames_col = tileplot_colorcode(fit_list=fit_list, nx_target=nx_target, ny_target=ny_target,
                                  outlier_threshold=outlier_threshold_color_plot,
                                  output_filename_stem=os.path.join(output_dir, output_filename_tileplot_color))

In [ ]:
def edges_to_centers(x):
    """
    Convert array edges to centers
    """
    return 0.5 * (x[1:] + x[:-1])

def tileplot(x_min_plot,x_max_plot,y_min_plot,y_max_plot,x_size,y_size,
             histogram_matrices,fitted_data, plots, fit_list, output_filename='tileplot_curves'):

    fig, ax = plt.subplots(x_size, y_size, figsize=(12, 12))

    labels = ["Without load", "Elastic deformation"]
    markers = ["o", "s"]
    # Plot some fake data so that we can have larger markers in the legend
    ax[0][0].plot([0, 1], [0, 1], ls="None", marker=markers[0], markersize=4, label=labels[0], color="C0")
    ax[0][0].plot([0, 1], [0, 1], ls="None", marker=markers[1], markersize=4, label=labels[1], color="C1")

    # All bins are identical in a dataset
    x0_nostrain = edges_to_centers(histogram_matrices[0].coords["wavelength"].values)
    
    # if the full inital sample was taken and no grouping was done
    if histogram_matrices[0].coords["spectrum"].shape[0] == 1:
        # x and y values of inital state, samples without strain
        y0_nostrain = histogram_matrices[0]["spectrum", 0].values

    plots_counter = 0
    for row in range(y_size):
        for col in range(x_size):
            # for running workspace index
            ws_index = plots[plots_counter][0]

            # plot if the spectrum is not masked
            if not histogram_matrices[1].masks["sample_elastic"]["spectrum", ws_index]:

                # if the whole inital-sample-state is just one spectrum, always plot that.
                if histogram_matrices[0].coords["spectrum"].shape[0] == 1:
                    # Plot no-strain curve, inital state of sample
                    ax[row][col].plot(x0_nostrain, y0_nostrain, ls="None", marker= "o", markersize=1)
                else: # else plot each of the grouped spectra accordingly
                    y0_nostrain = histogram_matrices[0].extractY()[ws_index]
                    ax[row][col].plot(x0_nostrain, y0_nostrain, ls="None", marker= "o", markersize=1)

                # Plot curves of other states given in histogram_matrices
                for k in range(len(histogram_matrices)):
                    if k != 0:
                        x0 = edges_to_centers(histogram_matrices[k].extractX()[ws_index])
                        y0 = histogram_matrices[k].extractY()[ws_index]
                        ax[row][col].plot(x0, y0, ls="None", marker=markers[k], markersize=1)

                # plotting the fitted spectra:
                x1 = edges_to_centers(fitted_data[0][plots_counter][1])
                x2 = edges_to_centers(fitted_data[1][plots_counter][1])
                ax[row][col].plot(x1, fitted_data[0][plots_counter][2], label="{} fit".format(labels[0]), lw=1, zorder=3)
                ax[row][col].plot(x2, fitted_data[1][plots_counter][2], label="{} fit".format(labels[1]), lw=1, zorder=3)

                # Set axis limits
                ax[row][col].set_xlim([x_min_plot, x_max_plot])
                ax[row][col].set_ylim([y_min_plot, y_max_plot])
                if col == x_size - 1:
                    ax[row][col].yaxis.tick_right()
                else:
                    ax[row][col].set_yticklabels([])
                if row == 0:
                    ax[row][col].xaxis.tick_top()
                else:
                    ax[row][col].set_xticklabels([])
                ax[row][col].tick_params(axis="x", direction="in", bottom=True, top=True)
                ax[row][col].tick_params(axis="y", direction="in", left=True, right=True)

                if row == y_size - 1:
                    ax[row][col].text(0.0, -0.1, "{:.1f}".format(0.1 * col),
                                    ha='center',va='top', transform=ax[row][col].transAxes)
                if col == 0:
                    ax[row][col].text(-0.1, 0.0, "{:.1f}".format(0.1*(y_size - row - 1)),
                                    ha='right',va='center', transform=ax[row][col].transAxes)

                if fit_list[plots_counter][-1]: # usual sign for lattice strain is \\varepsilon
                    ax[row][col].text(0.1, 0.8, "${:.3f}$".format(fit_list[plots_counter][-2]),
                                    ha='left',va='top', transform=ax[row][col].transAxes, fontsize=4)
            plots_counter += 1

    # Last bin edge for xy coordinates
    ax[-1][-1].text(1.0, -0.1, "{:.1f}".format(0.1 * x_size),
                    ha='center',va='top', transform=ax[-1][-1].transAxes)
    ax[0][0].text(-0.1, 1.0, "{:.1f}".format(0.1 * y_size),
                  ha='right',va='center', transform=ax[0][0].transAxes)
    # Display only one legend
    ax[0][0].legend(loc=(0, 1.8), ncol=4)
    fig.text(0.5, 0.905, "Wavelength $[\mathrm{\AA}]$")
    fig.text(0.94, 0.5, "Counts", rotation=90, ha='center',va='center')
    fig.text(0.5, 0.075, "X position [m]")
    fig.text(0.09, 0.5, "Y position [m]", rotation=90, ha='center',va='center')
    # Remove white space between subplots
    fig.subplots_adjust(wspace=0.0, hspace=0.0)
    print(('Saving {}.pdf.').format(output_filename))
    fig.savefig("{}.pdf".format(output_filename), bbox_inches="tight")
    fig.savefig("{}.png".format(output_filename), bbox_inches="tight")
    return output_filename

output_filename_tileplot = '{:03d}_tileplot_curves_{}xy_{}usbin_{}thresh_initintegr{}'.format(
                        measurement_number,grouping_number,tbin_width,masking_threshold, str(integrating_of_sample_in_inital_state))

# Plotting: y_min_plot=200,y_max_plot=270 for 18x18, y_min_plot=90,y_max_plot=120 for 27x27.
plotting_win = [(i[1]*(1-.135), i[1]*(1+.135)) for i in Bragg_edges_FCC]
for i in range(len(Bragg_edges_FCC)):
    output_filename_tileplot_new = '{}_{:.3f}A_({}{}{})'.format(output_filename_tileplot, Bragg_edges_FCC[i][1]/2.,
                                                        Bragg_edges_FCC[i][0][0], Bragg_edges_FCC[i][0][1], Bragg_edges_FCC[i][0][2])
    win = plotting_win[i]
    fignames_curve = tileplot(x_min_plot=win[0],x_max_plot=win[1],y_min_plot=65610./(nx_target*ny_target),y_max_plot=87480./(nx_target*ny_target), # remove 0.9 later
        x_size=nx_target,y_size=ny_target,
        histogram_matrices=histogram_matrices,
        fitted_data=[spectrum_list_fitted_sample[i], spectrum_list_fitted_sample_elastic[i]],
        plots=plots, fit_list=fit_list[i], output_filename=os.path.join(output_dir, output_filename_tileplot_new))